<a href="https://colab.research.google.com/github/RPGraciotti/BootCampAlura/blob/main/Modulo_5/Aula_4_valida%C3%A7%C3%A3o_cruzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

dados = pd.read_excel("https://github.com/alura-cursos/covid-19-clinical/blob/main/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true")
### também posso usar o link no meu repositório

def preenche_tabela(dados):
    features_continuas_colunas = dados.iloc[:, 13:-2].columns
    features_continuas = dados.groupby("PATIENT_VISIT_IDENTIFIER", as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')
    features_categoricas = dados.iloc[:, :13]
    saida = dados.iloc[:, -2:]
    dados_finais = pd.concat([features_categoricas, features_continuas, saida], ignore_index=True,axis=1)
    dados_finais.columns = dados.columns
    return dados_finais

dados_limpos = preenche_tabela(dados)
a_remover = dados_limpos.query("WINDOW=='0-2' and ICU==1")['PATIENT_VISIT_IDENTIFIER'].values
dados_limpos = dados_limpos.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")
dados_limpos = dados_limpos.dropna()

def prepare_window(rows):
    if(np.any(rows["ICU"])):
        rows.loc[rows["WINDOW"]=="0-2", "ICU"] = 1
    return rows.loc[rows["WINDOW"] == "0-2"]

dados_limpos = dados_limpos.groupby("PATIENT_VISIT_IDENTIFIER").apply(prepare_window)
dados_limpos.AGE_PERCENTIL = dados_limpos.AGE_PERCENTIL.astype("category").cat.codes # transformando em variaveis categoricas


np.random.seed(73246)

x_columns = dados.columns
y = dados_limpos["ICU"]
x = dados_limpos[x_columns].drop(["ICU","WINDOW"], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)
modelo = DummyClassifier()
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


0.48863636363636365

In [ ]:
modelo = LogisticRegression(max_iter=10000)
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

0.6818181818181818

In [ ]:
modelo_arvore = DecisionTreeClassifier()

modelo_arvore.fit(x_train, y_train)
pred_arvore = modelo_arvore.predict(x_test)
accuracy_score(y_test, pred_arvore)


0.6590909090909091

In [ ]:

def roda_n_modelos(modelo, dados, n):

  x_columns = dados.columns
  y = dados["ICU"]
  x = dados[x_columns].drop(["ICU","WINDOW"], axis=1)
  auc_lista = []

  for _ in range(n):

    x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)
    modelo.fit(x_train, y_train)
    prob_predic = modelo.predict_proba(x_test)
    auc = roc_auc_score(y_test, prob_predic[:,1])
    auc_lista.append(auc)

  auc_medio = np.mean(auc_lista)
  auc_std = np.std(auc_lista)
  print(f"AUC {auc_medio}")
  print(f"Intervalo {auc_medio - 2 * auc_std} - {auc_medio + 2 * auc_std}") # confidence interval
  #plt.hist(auc_lista)

# Aula 4

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

In [ ]:
cv = StratifiedKFold(n_splits = 5, shuffle = True)
 cross_validate(modelo, x, y, cv = cv) # comumente utilizado entre 5 e 10, tambem existe um tradeoff ai entre tamanho do dataset

{'fit_time': array([0.27184939, 0.69760394, 0.54343438, 0.63304949, 0.46535039]),
 'score_time': array([0.00229192, 0.0024209 , 0.00223827, 0.00232196, 0.00225115]),
 'test_score': array([0.67605634, 0.6056338 , 0.71428571, 0.72857143, 0.77142857])}

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

In [28]:
cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10)
cross_validate(modelo, x, y, cv = cv)

{'fit_time': array([0.67148232, 0.27660632, 0.66390657, 0.78517103, 0.5679605 ,
        0.56971288, 0.22172379, 0.15842915, 0.62874794, 0.2357347 ,
        0.56823039, 0.51588583, 0.64804792, 0.2536335 , 0.60821104,
        0.64481068, 0.52425599, 0.23909163, 0.33124876, 0.60460043,
        0.61804438, 0.23728061, 0.55578327, 0.67018533, 0.50881815,
        0.5141356 , 0.6903224 , 0.55404329, 0.62101579, 0.65475845,
        0.575068  , 0.60293531, 0.55347681, 0.60208178, 0.34177971,
        0.61364174, 0.22127342, 0.16210604, 0.28454781, 0.6120193 ,
        0.56020832, 0.61032343, 0.56884384, 0.59314251, 0.19878364,
        0.17300677, 0.58380723, 0.66430211, 0.6838398 , 0.67226434]),
 'score_time': array([0.00238752, 0.00228   , 0.002321  , 0.00228548, 0.00228262,
        0.0022316 , 0.00241852, 0.00223994, 0.00221658, 0.00220656,
        0.00227094, 0.00233269, 0.00229645, 0.00226402, 0.00221157,
        0.0022788 , 0.00229526, 0.00226092, 0.0023222 , 0.00231171,
        0.00228405, 

In [42]:
def roda_modelo_cv(modelo, dados, n_splits, n_repeats):

    np.random.seed(1231234) # lembrando sempre que é para valor didático
    dados = dados.sample(frac = 1).reset_index(drop = True) # embaralha
    x_columns = dados.columns
    y = dados["ICU"]
    x = dados[x_columns].drop(["ICU","WINDOW"], axis=1)

    cv = RepeatedStratifiedKFold(n_splits = n_splits, n_repeats = n_repeats)
    resultados = cross_validate(modelo, x, y, cv = cv, scoring = "roc_auc")

    auc_medio = np.mean(resultados["test_score"])
    auc_std = np.std(resultados["test_score"])

    print(f"AUC Médio {auc_medio}") 
    print(f"Intervalo {auc_medio - (2 * auc_std)} - {auc_medio + (2 * auc_std)}")


In [43]:
roda_modelo_cv(modelo, dados_limpos, 5, 10)

AUC Médio 0.7575511875511876
Intervalo 0.6366273412799195 - 0.8784750338224556


In [44]:
roda_n_modelos(modelo, dados_limpos, 50)

AUC 0.7611728074727554
Intervalo 0.6552383213129391 - 0.8671072936325717


# Desafio 01 - Testar outros splitter classes e observar as diferenças